In [ ]:
import json
import random

import matplotlib.pyplot as plt
import numpy as np

from model_graph import ModelGraph
from simulator import Simulator

# Demo run

In [ ]:
graph_dict = json.load(open("graph_dict.json"))

In [ ]:
sim = Simulator(graph_dict=graph_dict)
sim.run()

In [ ]:
# Code to show graph

aggregated = np.sum(sim.state_var, axis=2)

fig, ax = plt.subplots()
ax.plot(sim.time_series, aggregated[:, sim.c_idx["S"]], "g")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["E"]], "r")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["I"]], "b")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["R"]], "purple")
ax.legend(["Susceptibles", "Exposed", "Infected", "Recovered"], loc="best") 
plt.show()

In [ ]:
# Code to show graph

fig, ax = plt.subplots()
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 0], "g")
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 1], "r")
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 2], "b")
ax.legend(["Age group 0-14", "Age group 15-59", "Age group 60+"], loc="best") 
plt.show()

# Multiple runs from a given point

In [ ]:
sim_original = sim

In [ ]:
time = np.array(sim_original.time_series).searchsorted(40)

In [ ]:
aggregated = np.sum(sim_original.state_var, axis=2)

fig, ax = plt.subplots()
ax.plot(sim_original.time_series[:time], aggregated[:time, sim_original.c_idx["I"]], "b")
ax.legend(["Infected"], loc="best") 
plt.show()

In [ ]:
ts_list = []
i_list = []
for i in range(3):
    sim_1 = Simulator(graph_dict=graph_dict)
    sim_1.init_vals = sim_original.state_var[time]
    sim_1.state_var = [sim_1.init_vals]
    sim_1.last_state = sim_original.state_var[time]
    sim_1.create_transition_matrix()
    sim_1.create_transmission_matrix()
    sim_1.run()
    
    aggregated = np.sum(sim_1.state_var, axis=2)
    
    ts_list.append(sim_1.time_series[:])
    i_list.append(aggregated[:, sim_1.c_idx["I"]])

In [ ]:
fig, ax = plt.subplots()
aggregated = np.sum(sim_original.state_var, axis=2)
ax.plot(sim_original.time_series[:time], aggregated[:time, sim_original.c_idx["I"]], "b")
for i in range(len(ts_list)):
    ax.plot(sim_original.time_series[time]+np.array(ts_list[i]), i_list[i], "r")
plt.show()

# Contact reduction

In [ ]:
time1 = np.array(sim_original.time_series).searchsorted(25)
time2 = np.array(sim_original.time_series).searchsorted(45)

In [ ]:
sim_1 = Simulator(graph_dict=graph_dict)
sim_1.init_vals = sim_original.state_var[time2]
sim_1.state_var = [sim_1.init_vals]
sim_1.last_state = sim_original.state_var[time2]
sim_1.model.contact_matrix = sim_1.model.contact_matrix * 0.5
sim_1.create_transition_matrix()
sim_1.create_transmission_matrix()
sim_1.run_after_csocs()

In [ ]:
fig, ax = plt.subplots()
aggregated = np.sum(sim_original.state_var, axis=2)
ax.plot(sim_original.time_series[:], aggregated[:, sim_original.c_idx["I"]], "b")

aggregated = np.sum(sim_1.state_var, axis=2)
ax.plot(sim_original.time_series[time2]+np.array(sim_1.time_series[:]), aggregated[:, sim_1.c_idx["I"]], "r")

plt.show()